## Easy

Оценить с помощью доверительных интервалов:
- выживаемость всех пассажиров (оценить среднее по isSurvived)
- выживаемость женщин
- выживаемость людей с каютами первого класса

In [1]:
import pandas as pn
import numpy as np
import scipy
from scipy.stats import norm
from scipy.stats import t

Загрузим данные с титаника. 

In [2]:
titanic = pn.read_csv('../02_lesson/titanic/train.csv')

Расcчитаем дисперсию признака выживаемости, а также среднюю выживаемости:

In [3]:
std = titanic.Survived.std()
mean = titanic.Survived.mean()
n = titanic.Survived.count()
print('Дисперсия выживаемости: ', std)
print('Средняя выживаемости: ', mean)
print('Количество пассажиров: ', n)

Дисперсия выживаемости:  0.4865924542648585
Средняя выживаемости:  0.3838383838383838
Количество пассажиров:  891


Определим функцию нахождения доверительного интервала при известной дисперсии.

In [4]:
def get_confidence_interval(mean, std, n, alpha):
    h = norm.ppf(1 - alpha / 2) * std / np.sqrt(n)
    return mean - h, mean + h

Рассчитаем доверительный интервал при достоверности 0.05 (alpha=0.05)

In [5]:
confidence_interval = get_confidence_interval(mean, std, n, 0.05)
confidence_interval

(0.3518881082221273, 0.41578865945464033)

Ответ: вероятность выживаемости на Титанике составляет от 35% до 41% с 95% точностью.

Поробуем расчитать вероятность выживания на Титанике если бы дисперсия и среднее были бы неизвестны

Попробуем посчитать тот же доверительный интервал выживаемости, но уже только для женщин

In [14]:
titanicFemale = titanic[titanic.Sex == 'female']
std = titanicFemale.Survived.std()
mean = titanicFemale.Survived.mean()
n = titanicFemale.Survived.count()
print('Дисперсия выживаемости: ', std)
print('Средняя выживаемости: ', mean)
print('Количество пассажиров: ', n)

Дисперсия выживаемости:  0.43821120391621965
Средняя выживаемости:  0.7420382165605095
Количество пассажиров:  314


In [15]:
confidence_interval = get_confidence_interval(mean, std, n, 0.05)
confidence_interval

(0.693568916951577, 0.790507516169442)

В 95% случаев выживаемость женщин находится в пределах от 69% до 79%
Расчитывать по альтернативной формуле не будем.

Теперь расчитаем выживаемость для пассажиров проплывающих в каюте первого класса

In [17]:
titanic1c = titanic[titanic.Pclass == 1]
std = titanic1c.Survived.std()
mean = titanic1c.Survived.mean()
n = titanic1c.Survived.count()
print('Дисперсия выживаемости: ', std)
print('Средняя выживаемости: ', mean)
print('Количество пассажиров: ', n)

Дисперсия выживаемости:  0.48402561132684707
Средняя выживаемости:  0.6296296296296297
Количество пассажиров:  216


In [18]:
confidence_interval = get_confidence_interval(mean, std, n, 0.05)
confidence_interval

(0.5650806238220164, 0.6941786354372429)

Доверительный интервал выживаемости пассажиров первого класса составляет от 56 до 69 процентов

## Normal

- Проверить гипотезу о том, что женщины выживают чаще мужчин с уровнем значимости 0.05
- Проверить гипотезу о том, что дети выживают чаще, чем взрослые

Для проверки гипотезы будем использовать критерий Стьюдента, для оценки средних двух выборок. В качестве выборки возьмем фактор выживаемости. Если средняя выживаемость женщин будет значимо отличатся от средней выживаемости мужчин, значит наша гипотеза верна. Критерий Стьюдента в качестве нулевой гипотезы использует предположение о том, что выборки равны. Таким образом, наша гипотеза является альтернативной, и для ее подтверждения нам необходимо выполнить тест с результатом достоверности p-value меньше 0.05 (среднее двух выборок равно с низкой степенью достоверности)

In [8]:
male = titanic[titanic.Sex == 'male']
female = titanic[titanic.Sex == 'female']
x, p_value = scipy.stats.ttest_ind(male.Survived.array, female.Survived.array, alternative = 'less')

if p_value < 0.05:
    print('Альтернативная гипотеза верна, p_value: ', p_value, ' Средняя выживаемости мужчин меньше средней выживаемости женщин')
else:
    print('Основная гипотеза верна, p_value: ', p_value, ' Средние выживаемости  с большой долей вероятностью равны')

Альтернативная гипотеза верна, p_value:  7.030330654401297e-70  Средняя выживаемости мужчин меньше средней выживаемости женщин


Сделаем ручной расчет. Переведем двухсторонюю гипотезу в односторонюю

In [21]:
male = titanic[titanic.Sex == 'male']
female = titanic[titanic.Sex == 'female']
x, p_value = scipy.stats.ttest_ind(male.Survived.array, female.Survived.array)
# просто делим на два
p_value = p_value / 2.0
if p_value < 0.05:
    print('Альтернативная гипотеза верна, p_value: ', p_value, ' Средняя выживаемости мужчин меньше средней выживаемости женщин')
else:
    print('Основная гипотеза верна, p_value: ', p_value, ' Средние выживаемости  с большой долей вероятностью равны')

Альтернативная гипотеза верна, p_value:  7.030330654401297e-70  Средняя выживаемости мужчин меньше средней выживаемости женщин


Значения p_value оказались равными.

Теперь проверим гипотезу о том, что выживаемость детей выши выживаемости взрослых

In [27]:
young = titanic[titanic.Age < 18]
adult = titanic[titanic.Age >= 18]
x, p_value = scipy.stats.ttest_ind(young.Survived.array, adult.Survived.array)
# просто делим на два
p_value = p_value / 2.0
if p_value < 0.05:
    print('Альтернативная гипотеза верна, p_value: ', p_value, ' Средняя выживаемости детей выше средней выживаемости взрослых')
else:
    print('Основная гипотеза верна, p_value: ', p_value, ' Средние выживаемости  с большой долей вероятностью равны')

Альтернативная гипотеза верна, p_value:  0.0007919436725484551  Средняя выживаемости детей выше средней выживаемости взрослых


Хотя альтернативная гипотеза в отношении детей равна, и их выживаемость выше выживаемости взрослых, тем не менее, значение p_value ближе к критической отметки 0.05 чем p_value выборок выживаемости женщин против мужчин. Таким образом можно сказать, что большее влияние на выживаемость оказывает пол пассажира, а не его возраст.